In [1]:
# Importing required libraries

import pandas as pd
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
from IPython.display import display, clear_output
import io
from tabulate import tabulate
import base64

In [2]:
from ipywidgets import HTML
from IPython.display import display

In [3]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix

In [4]:
# File Upload

Input = widgets.FileUpload(
        accept = '.csv',
        multiple = False,
        layout = widgets.Layout(width = '50%')
    )

inputs1 = widgets.HBox([Input], 
                       layout = widgets.Layout(
                        justify_content = 'center',
                        width = '100%'
                        ))

In [5]:
# Calculate button

clickbutton = widgets.Button(
            description = 'Enter Weights and Impacts',
            layout = widgets.Layout(width = '50%')
        )


B1 = widgets.HBox([clickbutton], 
                       layout = widgets.Layout(
                        justify_content = 'center',
                        width = '100%'
                        ))

calculate = widgets.Button(
            description = 'Calculate',
            layout = widgets.Layout(width = '50%')
        )

In [6]:
def perform_Topsis(df, weights, impacts):

    ws = 0
    for i in range(len(weights)):
        try:
            weights[i] = float(weights[i])
            ws+=weights[i]
        except:    
            print('\nWeights must be numeric and comma separated, Exiting....')
            exit(0)

    weights = [(i/ws) for i in weights]

    for i in impacts:
        if(i not in ['+','-']):
            print('\nImpacts must be either positive or negative and comma separated, Exiting....')
            exit(0)
    
    if(len(df.columns)<3):
        print('\nMinimum 3 columns required, Exiting....')
        exit(0)

    if(df.isnull().values.any()):
        print('\nNull Value Detected, Exiting....')
        exit(0)

    for i in range(len(df)):
        for j in range(1, len(df.columns)):
            try:
                df.iloc[i, j] = float(df.iloc[i, j])
            except:
                print('\nNot numeric value detected in column number', j+1, ', Exiting....')
                exit(0)
            
    if(len(impacts)!=len(df.columns)-1):
        print('\n',len(df.columns)-1,' impacts were required instead ', len(impacts),' were given, Exiting....')
        exit(0)

    if(len(weights)!=len(df.columns)-1):
        print('\n',len(df.columns)-1,' weights were required instead ', len(weights),' were given, Exiting....')
        exit(0)
    
    temp = df.copy()
    temp['Splus'] = 0
    temp['Sminus'] = 0
    ib = []
    iw = []

    for i in range(1, len(temp.columns)-2):
        SQ = (sum(temp.iloc[:,i]**2))**0.5
        temp.iloc[:,i] = ((temp.iloc[:,i]/SQ)*weights[i-1])
        if(impacts[i-1] == '+'):
            ib.append(max(temp.iloc[:,i]))
            iw.append(min(temp.iloc[:,i]))
        else:
            ib.append(min(temp.iloc[:,i]))
            iw.append(max(temp.iloc[:,i]))

    for i in range(len(temp)):
        temp.iloc[i, len(temp.columns)-2] = ((sum((temp.iloc[i,1:len(temp.columns)-2]-ib)**2))**0.5)
        temp.iloc[i, len(temp.columns)-1] = ((sum((temp.iloc[i,1:len(temp.columns)-2]-iw)**2))**0.5)

    df['Topsis Score'] = 0
    df['Rank'] = 0

    for i in range(len(temp)):
        SUM = temp.iloc[i, len(temp.columns)-1] + temp.iloc[i, len(temp.columns)-2] 
        df.iloc[i, len(temp.columns)-2] = temp.iloc[i, len(temp.columns)-1]/SUM

    df['Rank'] = df['Topsis Score'].rank(ascending = False, method='dense')
    
    return df

In [7]:
dfields = []
radios = []

In [8]:
def displayfields(event):
    with output1:
        clear_output()
        df = pd.read_csv(io.BytesIO(Input.value[list(Input.value.keys())[0]]['content']))
        col_list = list(df.columns)
        col_list = col_list[1:]

        for i in col_list:
            dfields.append(widgets.FloatText(value = 0.0, description = str(i), disabled = False, layout = widgets.Layout(width = '25%')))
            radios.append(widgets.RadioButtons(options = ['+', '-'], description = str(i), disabled = False, layout = widgets.Layout(width = '25%')))
            
        display(widgets.HBox(dfields, layout = widgets.Layout(
                            justify_content = 'center',
                            width = '100%'
                            )))
        display(widgets.HBox(radios, layout = widgets.Layout(
                            justify_content = 'center',
                            width = '100%'
                            )))
        display(widgets.HBox([calculate], layout = widgets.Layout(
                            justify_content = 'center',
                            width = '100%'
                            )))

In [9]:
# On button click function definition

def on_button_clicked(event):
    with output2:
        clear_output()
        df = pd.read_csv(io.BytesIO(Input.value[list(Input.value.keys())[0]]['content']))
        weights = []
        impacts = []
        
        for i in dfields:
            weights.append(float(i.value))
            
        for i in radios:
            impacts.append(str(i.value))
        
        ANS = perform_Topsis(df, weights, impacts)
        print(tabulate(ANS, headers='keys', tablefmt='psql', showindex = False))
        res = ANS.to_csv(index = False)
        #FILE
        filename = 'Output.csv'
        b64 = base64.b64encode(res.encode())
        payload = b64.decode()
        #BUTTONS
        html_buttons = '''<html>
        <head>
        <meta name="viewport" content="width=device-width, initial-scale=1">
        </head>
        <body>
        <br>
        <br>
        <center><a download="{filename}" href="data:text/csv;base64,{payload}" download>
        <button class="p-Widget jupyter-widgets jupyter-button widget-button mod-warning">Download File</button>
        </a></center>
        </body>
        </html>
        '''
        html_button = html_buttons.format(payload=payload,filename=filename)
        display(HTML(html_button))

In [10]:
clickbutton.on_click(displayfields)
calculate.on_click(on_button_clicked)

In [11]:
# Output widget

output1 = widgets.Output()
output2 = widgets.Output()

OP2 = widgets.HBox([output2], layout = widgets.Layout(justify_content = 'center', width = '100%'))

In [12]:
# Line breaker

text_0 = widgets.HTML(value = "<h1></h1>", 
                     layout = widgets.Layout(
                     align_items = 'center',
                     )
                    )

In [13]:
# Heading

text_1 = widgets.HTML(value = """
<center style="font-size: 100px"><b>TOPSIS</b></center>
""")

# Sub heading

text_2 = widgets.HTML(value = "<h3><center>Technique for Order Preference by Similarity to Ideal Solution</center></h3>")

headings = widgets.VBox([text_1, text_2, text_0])

# About dataset

text_4 = widgets.HTML(value = """
<table style="width: 100%; border-collapse: collapse;" border="1" cellpadding="20">
<tbody>
<tr>
<td style="width: 100%;">
<p style="text-align: justify;"><strong class="iy fz">TOPSIS</strong>, known as&nbsp;<strong class="iy fz">T</strong>echnique for&nbsp;<strong class="iy fz">O</strong>rder of&nbsp;<strong class="iy fz">P</strong>reference by&nbsp;<strong class="iy fz">S</strong>imilarity to&nbsp;<strong class="iy fz">I</strong>deal&nbsp;<strong class="iy fz">S</strong>olution, is a multi-criteria decision analysis method. It compares a set of alternatives based on a pre-specified criterion. The method is used in the business across various industries, every time we need to make an analytical decision based on collected data. The mysterious logic of&nbsp;<strong class="iy fz">TOPSIS</strong>&nbsp;is based on the concept that the chosen alternative should have the shortest geometric distance from the best solution and the longest geometric distance from the worst solution.</p>
</td>
</tr>
</tbody>
</table>
""")

text_5 = widgets.HTML(value = """
<center style="font-size: 40px"><b>TOPSIS Algorithm</b></center>""")

# Image 1

image1 = open("image1.png", "rb")
image1 = image1.read()
image1 = widgets.Image(
    value = image1,
    format = 'png',
    width = 800,
    height = 600,
)


# Image 2

image2 = open("image2.png", "rb")
image2 = image2.read()
image2 = widgets.Image(
    value = image2,
    format = 'png',
    width = 800,
    height = 600,
)

# Image 3

image3 = open("image3.png", "rb")
image3 = image3.read()
image3 = widgets.Image(
    value = image3,
    format = 'png',
    width = 800,
    height = 600,
)

# Image 4

image4 = open("image4.png", "rb")
image4 = image4.read()
image4 = widgets.Image(
    value = image4,
    format = 'png',
    width = 800,
    height = 600,
)

# Image 5

image5 = open("image5.png", "rb")
image5 = image5.read()
image5 = widgets.Image(
    value = image5,
    format = 'png',
    width = 800,
    height = 600,
)

# Image 6

image6 = open("image6.png", "rb")
image6 = image6.read()
image6 = widgets.Image(
    value = image6,
    format = 'png',
    width = 800,
    height = 600,
)

# Image 7

image7 = open("image7.png", "rb")
image7 = image7.read()
image7 = widgets.Image(
    value = image7,
    format = 'png',
    width = 800,
    height = 600,
)

ImageHbox1 = widgets.HBox([image1],
                        layout = widgets.Layout(
                        justify_content = 'center',
                        )
                    )

ImageHbox2 = widgets.HBox([image2],
                        layout = widgets.Layout(
                        justify_content = 'center',
                        )
                    )


ImageHbox3 = widgets.HBox([image3],
                        layout = widgets.Layout(
                        justify_content = 'center',
                        )
                    )


ImageHbox4 = widgets.HBox([image4],
                        layout = widgets.Layout(
                        justify_content = 'center',
                        )
                    )


ImageHbox5 = widgets.HBox([image5],
                        layout = widgets.Layout(
                        justify_content = 'center',
                        )
                    )


ImageHbox6 = widgets.HBox([image6],
                        layout = widgets.Layout(
                        justify_content = 'center',
                        )
                    )


ImageHbox7 = widgets.HBox([image7],
                        layout = widgets.Layout(
                        justify_content = 'center',
                        )
                    )

text_6 = widgets.HTML(value = """<center style="font-size: 40px"><b>Live Demo</b></center>""")

In [14]:
# Displaying rendered Web Page

page = widgets.VBox([text_0, text_1, text_0, text_2, text_0, text_4, text_0, text_0, text_0, text_5, text_0, ImageHbox1, text_0, text_0, ImageHbox2, text_0, text_0, ImageHbox3, text_0, text_0, ImageHbox4, text_0, text_0, ImageHbox5, text_0, text_0, ImageHbox6, text_0, text_0, ImageHbox7, text_0, text_6, text_0, text_0, inputs1, text_0, text_0, text_0, B1, text_0, output1, OP2, text_0, text_0, text_0])
display(page)